This notebook provides some top-view logic to train the object detector (YOLO family of models) and the segmentor separately. This step is optional, both the segmentor and detector have already been trained and have checkpoints available. If you don't wish to train the segmentor and detector again, you can go to *[train_combined.ipynb](train_combined.ipynb)* notebook.

# 🚀 Train object detector

For more information about the training process and parameters, check the https://docs.ultralytics.com/modes/train/#train-settings.

For example, to download yolov8 original checkpoints, use https://docs.ultralytics.com/models/yolov8/#supported-tasks-and-modes.

In [ ]:
from ultralytics import YOLO, RTDETR
import sys
import os

yolo_dataset_path = f"../../data/xami_dataset_YOLO/" # replace with path to YOLO dataset
data_yaml_path = yolo_dataset_path+'data.yaml'
device = 3

# Train YOLOv8-segm model
model_checkpoint = 'yolov8n-seg.pt'
model = YOLO(model_checkpoint) 

project = f"yolov8-segm-0" 
name = model_checkpoint.replace('.pt', '') 

results = model.train(data=data_yaml_path,
                      project=project,
                      name=name,
					  task='detect',
                      epochs=300,
                      patience=0, # patience=0 disables early stopping
                      batch=16,
                      imgsz=512,
                      device=device,
					  hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
					  lr0=0.0006,
					  dropout=0.2,
					  mask_ratio = 1,
					  mosaic=0,
					  # cos_lr=True,
					  box=1.0,
					  cls=0.8,
					  # label_smoothing=0.1,
                      # augment=True, 
                      # freeze layers
                     )

# 🚀 Train segmentor

In [ ]:
import yaml
from pprint import pprint

# create a training configuration
config_file_path = './xami_model/train/segmentor_config.yaml'
with open(config_file_path, 'r') as file:
     config = yaml.safe_load(file)

print('Initial config:')
pprint(config)
config['device_id'] = 1
config['input_dir'] = './data/xami_dataset/'
config['mobile_sam_checkpoint'] = './xami_model/train/weights/sam_weights/original_mobile_sam.pt'

# save the configuration
with open(config_file_path, 'w') as file:
     yaml.safe_dump(config, file)
print(f"Updated configuration saved to {config_file_path}.")

**run training**

In [ ]:
# replace the python executable with the environment where xami packages are installed
!/root/miniforge3/envs/xami_model_env_2/bin/python ./train_sam.py ./segmentor_config.yaml